In [75]:
import json
import pandas as pd
import numpy as np
import itertools

In [64]:
DUPE_CSV_PATH = '../graph/pairwise-sim.csv'
DEDUPE_CSV_PATH = '../graph/training-data.csv'

df = pd.read_csv(DUPE_CSV_PATH)


In [71]:
averager = {}

def add_pair(a, b, weight):
    if a not in averager:
        averager[a] = {}
    if b not in averager[a]:
        averager[a][b] = []
    averager[a][b].append(weight)

for index, row in df.iterrows():
    a_or_b = row["project_a"] > row["project_b"]
    if a_or_b:
        add_pair(row["project_a"], row["project_b"], row["weight_a"])
    else:
        add_pair(row["project_b"], row["project_a"], row["weight_b"])


In [81]:
deduped = []

for _, (a, inner) in enumerate(averager.items()):
    for _, (b, weights) in enumerate(inner.items()):
        weight_a = float(np.average(weights))
        weight_b = 1-weight_a
        deduped.append([a, b, weight_a, weight_b])

deduped_df = pd.DataFrame(deduped, columns=["project_a", "project_b", "weight_a", "weight_b"])

In [82]:
deduped_df.to_csv(DEDUPE_CSV_PATH)